In [1]:
pip install pydantic-ai ollama logfire devtools googlesearch-python

Note: you may need to restart the kernel to use updated packages.


In [2]:
import re
import os
from dataclasses import dataclass
from typing import Any
from datetime import datetime
import gradio as gr
import nest_asyncio
from dotenv import load_dotenv
from httpx import AsyncClient
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from datetime import datetime
from pydantic_ai import Agent, ModelRetry, RunContext
from pydantic_ai.models.openai import OpenAIModel
from pydantic_ai.providers.openai import OpenAIProvider
from datetime import datetime, timedelta
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from googlesearch import search
# ---------- Setup ----------
load_dotenv()
nest_asyncio.apply()

ollama_model = OpenAIModel(
    model_name='qwen2.5:latest',
    provider=OpenAIProvider(base_url='http://localhost:11434/v1')
)


@dataclass
class Deps:
    client: AsyncClient
    weather_api_key: str | None


agent = Agent(
    ollama_model,
    system_prompt=(
        'You are a smart assistant. You can access tools:\n'
        '- get_weather(city: str): Returns weather info.\n'
        '- calculate_expression(expression: str): Evaluates math.\n'
        '- tripPlanner(start_date: date, end_date: date, location: str): Trip planner.\n'
        '- add_to_google_calendar(summary, location, start_date, end_date): Adds to Google Calendar.\n'
        '- get_upcoming_events(start_date, end_date): Retrieves upcoming events.\n'
        '- search(query: str): Searches the web and gives a structured response.\n'
        'Use these tools when the user requests relevant information.'
    ),
    deps_type=Deps,
    retries=2,
    instrument=True,
)


trip_agent = Agent(
    ollama_model,
    output_type=str,)


In [3]:
import httpx


async def fetch_wikipedia_summary(location: str) -> str:
    try:
        async with httpx.AsyncClient() as client:
            response = await client.get(f"https://en.wikipedia.org/api/rest_v1/page/summary/{location}")
            if response.status_code == 200:
                return response.json().get("extract", "")
    except:
        return ""


def is_valid_expression(expr: str) -> bool:
    return bool(re.fullmatch(r"[0-9+\-*/().\\s]+", expr.strip()))


@agent.tool
async def get_weather(context: RunContext[Deps], city: str) -> str:
    """
    Fetches the current weather temperature for a specified city using the WeatherAPI.

    Args:
        city (str): The name of the city for which to fetch the weather.

    Returns:
        str: A message containing the current temperature in Celsius for the specified city, 
             or an error message if the city is not found or the API response is invalid.

    Example:
        >>> get_weather("Dhaka")
        'The current temperature in Dhaka is 30°C.'
    """
    if not context.deps.weather_api_key:
        raise ValueError("Weather API key is not set.")

    url = f"http://api.weatherapi.com/v1/current.json?key={context.deps.weather_api_key}&q={city}"
    response = await context.deps.client.get(url)
    data = response.json()
    print(f"Tool Get_weather: {data}")
    return f"The weather in {city} is {data['current']['temp_c']}\u00b0C."


@agent.tool_plain
async def calculate_expression(expression: str) -> str:
    """
    Evaluates a mathematical expression provided as a string.

    Args:
        expression (str): A string containing a mathematical expression to be evaluated.

    Returns:
        str: The result of the evaluation as a string, or an error message if the evaluation fails.

    Example:
        >>> calculate_expression("2 + 3 * 4")
        '14'
        >>> calculate_expression("10 / 0")
        'Calculation error: division by zero'
    """
    if not is_valid_expression(expression):
        raise ModelRetry('This does not look like a valid math expression.')
    try:
        print(f"Tool Calculate_expression: {expression}")
        return str(eval(expression))
    except Exception as e:
        return f"Calculation error: {e}"


@agent.tool_plain
async def tripPlanner(start_date: str, end_date: str, location: str) -> str:
    try:
        start = datetime.strptime(start_date, "%Y-%m-%d")
        end = datetime.strptime(end_date, "%Y-%m-%d")
        duration = (end - start).days + 1
        today = datetime.now().date()
        if start.date() < today:
            return "❌ The start date cannot be in the past."
        if end.date() < today:
            return "❌ The end date cannot be in the past."
        if end.date() <= start.date():
            return "❌ The end date must be after the start date."

        if duration <= 0:
            return "Invalid date range. Please check the start and end dates."

        contexts = await fetch_wikipedia_summary(location)

        prompt = (
            f"Context:\n{contexts}\n\n"
            f"I'm planning a {duration}-day trip to {location}. "
            f"Give me the following:\n"
            f"- Famous attractions\n"
            f"- Food recommendations\n"
            f"- Fun activities\n"
            f"- A fun fact\n"
            f"- Estimated budget (in local currency)\n"
            f"Keep it concise and friendly."
        )

        response = await trip_agent.run(
            prompt,
        )
        print(f"toolPlanner")
        return response.output

    except ValueError:
        return "Please use YYYY-MM-DD format for dates."
    except Exception as e:
        return f"Error: {e}"


@agent.tool_plain
async def add_to_google_calendar(summary: str, location: str, start_date: str, end_date: str) -> str:
    """
    Adds an event to Google Calendar with the provided details.

    Args:
        summary (str): The title/summary of the event.
        location (str): The location of the event.
        start_date (str): The start date of the event in 'YYYY-MM-DD' format.
        end_date (str): The end date of the event in 'YYYY-MM-DD' format.

    Returns:
        str: A confirmation message with the event link, or an error message.

    Example:
        >>> add_to_google_calendar("Trip to Sylhet", "Sylhet", "2024-06-01", "2024-06-05")
        '✅ Trip added to your Google Calendar: https://calendar.google.com/event?eid=...'
    """

    # Validate dates are not in the past
    current_datetime = datetime.now()

    # Parse start and end dates
    start_datetime = datetime.strptime(
        f'{start_date}T09:00:00', '%Y-%m-%dT%H:%M:%S')
    end_datetime = datetime.strptime(
        f'{end_date}T17:00:00', '%Y-%m-%dT%H:%M:%S')

    if start_datetime < current_datetime:
        return "❌ The start date cannot be in the past."
    if end_datetime < current_datetime:
        return "❌ The end date cannot be in the past."
    if end_datetime <= start_datetime:
        return "❌ The end date must be after the start date."

    # Google Calendar API setup
    SCOPES = ['https://www.googleapis.com/auth/calendar.events']

    creds = None
    if os.path.exists('token.json'):
        creds = Credentials.from_authorized_user_file('token.json', SCOPES)
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                'credentials.json', SCOPES)
            creds = flow.run_local_server(port=0)
        with open('token.json', 'w') as token:
            token.write(creds.to_json())

    service = build('calendar', 'v3', credentials=creds)

    event = {
        'summary': summary,
        'location': location,
        'start': {
            'dateTime': f'{start_date}T09:00:00',
            'timeZone': 'Asia/Dhaka',
        },
        'end': {
            'dateTime': f'{end_date}T17:00:00',
            'timeZone': 'Asia/Dhaka',
        },
        'description': f'Trip to {location} from {start_date} to {end_date}. Auto-generated by AI.'
    }

    # Create the event on Google Calendar
    event = service.events().insert(calendarId='primary', body=event).execute()
    print(f"Event created: {event.get('htmlLink')}")

    return f"✅ Trip added to your Google Calendar: {event.get('htmlLink')}"


@agent.tool_plain
async def get_upcoming_events(start_date=None, end_date=None):
    """
    Retrieves upcoming Google Calendar events within a given date range if given else retrives next one month events from today.

    Args:
        start_date (str, optional): Start date in 'YYYY-MM-DD' format or 'today'. Defaults to today.
        end_date (str, optional): End date in 'YYYY-MM-DD' format. Defaults to 1 month from today.

    Returns:
        str: A list of upcoming events with date, time, and summary.

    Example:
        >>> get_upcoming_events("2024-06-01", "2024-06-15")
        '📅 June 2, 10:00 AM - Team Meeting\n📅 June 5, 3:00 PM - Doctor Appointment\n...'
    """

    SCOPES = ['https://www.googleapis.com/auth/calendar.readonly']

    # Auth setup
    creds = None
    if os.path.exists('token.json'):
        creds = Credentials.from_authorized_user_file('token.json', SCOPES)
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                'credentials.json', SCOPES)
            creds = flow.run_local_server(port=0)
        with open('token.json', 'w') as token:
            token.write(creds.to_json())

    service = build('calendar', 'v3', credentials=creds)

    # Default date range: today to 30 days later
    now = datetime.utcnow()

    if start_date == "today" or not start_date:
        time_min = now
    else:
        time_min = datetime.strptime(start_date, '%Y-%m-%d')

    if end_date:
        time_max = datetime.strptime(end_date, '%Y-%m-%d')
    else:
        time_max = now + timedelta(days=30)

    time_min_str = time_min.isoformat() + 'Z'
    time_max_str = time_max.isoformat() + 'Z'

    # Fetch events
    events_result = service.events().list(
        calendarId='primary',
        timeMin=time_min_str,
        timeMax=time_max_str,
        singleEvents=True,
        orderBy='startTime'
    ).execute()

    events = events_result.get('items', [])

    if not events:
        return "📭 No upcoming events found in the selected date range."

    result = []
    for event in events:
        start = event['start'].get('dateTime', event['start'].get('date'))
        start_dt = datetime.fromisoformat(start.replace('Z', '+00:00'))
        start_formatted = start_dt.strftime("📅 %B %d, %I:%M %p")
        summary = event.get('summary', '(No Title)')
        result.append(f"{start_formatted} - {summary}")

    print(f"Upcoming events:")
    return "\n".join(result)
'''
@agent.tool_plain
async def searchWeb(query: str) -> str:
    """
    Searches for a query using Google Search and returns the top 5 results with titles and descriptions.

    Args:
        query (str): The search query.

    Returns:
        str: Titles, URLs, and descriptions of the top 5 search results.
    """
    try:
        results = list(search(query, num_results=5, advanced=True))
        if results:
            print(f"Tool SearchWeb: {results}")
            formatted = "\n\n".join(
                f"{i+1}. {res.title}\n{res.url}\n{res.description}" for i, res in enumerate(results)
            )
            return f"Top 5 results:\n\n{formatted}"
        else:
            return "No results found."
    except Exception as e:
        return f"Error during search: {e}"'''

@agent.tool_plain
async def searchWeb(query: str) -> str:
    '''
    Searches for a query using Google Search and returns the top 5 results with titles and descriptions.'''
    try:
        apikey = os.getenv("cloudapi")
        search_engine_id = os.getenv("searchengine")
        async with httpx.AsyncClient() as client:
            response = await client.get(
                f"https://www.googleapis.com/customsearch/v1?key={apikey}&cx={search_engine_id}&q={query}"
            )
        data = response.json()
        if "items" in data:
            results = data["items"][:5]  # Limit to top 5 results
            formatted = "\n\n".join(
                f"{i+1}. {res['title']}\n{res['link']}\n{res['snippet']}" for i, res in enumerate(results)
            )
            print(f"Tool SearchWeb: {results}")
            return f"Top 5 results:\n\n{formatted}"
        else:
            return "No results found."
    except Exception as e:
        return f"Error during search: {e}"

In [4]:
# ---------- Gradio Setup ----------
chat_messages = []  # Global message history


async def chat_with_agent(message, history):
    weather_key = os.getenv('weatherapicom_API_KEY')
    async with AsyncClient() as client:
        deps = Deps(client=client, weather_api_key=weather_key)
        global chat_messages

        # Prepare conversation
        if not chat_messages:
            result = await agent.run(message, deps=deps)
        else:
            result = await agent.run(message, deps=deps, message_history=chat_messages)

        chat_messages = result.new_messages()

        # Process the result to check if a tool was called and format the output
        tool_output = result.output

        # If no tool was called, just return the assistant's output
        formatted_response = f"🤖 {tool_output}"

        return "", history + [[message, formatted_response]]


def reset_chat():
    global chat_messages
    chat_messages = []
    return [], ""


with gr.Blocks() as demo:
    gr.Markdown("## 🤖 Smart Assistant with Tools")

    chatbot = gr.Chatbot()
    msg = gr.Textbox(label="Ask me something...")
    clear = gr.Button("Clear Chat")

    msg.submit(chat_with_agent, [msg, chatbot], [msg, chatbot])
    clear.click(reset_chat, outputs=[chatbot, msg])
    demo.unload(reset_chat)

# ---------- Launch ----------
demo.launch(share=True)

/tmp/ipykernel_2887214/3320054510.py:37: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot()


* Running on local URL:  http://127.0.0.1:7860
* Running on public URL: https://03975b0172e8e36669.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Tool SearchWeb: [{'kind': 'customsearch#result', 'title': 'Bangladesh - Wikipedia', 'htmlTitle': '<b>Bangladesh</b> - Wikipedia', 'link': 'https://en.wikipedia.org/wiki/Bangladesh', 'displayLink': 'en.wikipedia.org', 'snippet': 'It is the eighth-most populous country in the world and among the most densely populated with a population of over 171 million within an area of 148,460 square\xa0...', 'htmlSnippet': 'It is the eighth-most populous country in the world and among the most densely populated with a population of over 171 million within an area of 148,460 square&nbsp;...', 'formattedUrl': 'https://en.wikipedia.org/wiki/Bangladesh', 'htmlFormattedUrl': 'https://en.wikipedia.org/wiki/<b>Bangladesh</b>', 'pagemap': {'hcard': [{'fn': "People's Republic of Bangladesh"}], 'metatags': [{'referrer': 'origin', 'og:image': 'https://upload.wikimedia.org/wikipedia/commons/thumb/f/f9/Flag_of_Bangladesh.svg/1200px-Flag_of_Bangladesh.svg.png', 'theme-color': '#eaecf0', 'og:image:width': '1200', 